In [1]:
import numpy as np
import pandas as pd
import time
import datetime

#### Ideas

- Do not include matches with no team from test set.
- Clean teams columns (e.g. strip)
- Add some coefficient of importantness to the tournaments
- Prefer newer matches
- Include home team advantage
- Skip city

In [2]:
path_matches = "Data/AI Match Results 150years_appended_WC2022.txt"

In [17]:
data_raw = pd.read_csv(path_matches)
display(data_raw.tail(50))


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44074,24.11.2022,Portugal,Ghana,NaN,NaN,FIFA World Cup 2022,Stadium 974,Qatar,True
44075,24.11.2022,Brazil,Serbia,NaN,NaN,FIFA World Cup 2022,Lusail Iconic,Qatar,True
44076,25.11.2022,Wales,Iran,NaN,NaN,FIFA World Cup 2022,Ahmad bin Ali,Qatar,True
44077,25.11.2022,Qatar,Senegal,NaN,NaN,FIFA World Cup 2022,Al Thumama,Qatar,False
44078,25.11.2022,Netherlands,Ecuador,NaN,NaN,FIFA World Cup 2022,Khalifa International,Qatar,True
44079,25.11.2022,England,United States,NaN,NaN,FIFA World Cup 2022,Al Bayt,Qatar,True
44080,26.11.2022,Tunisia,Australia,NaN,NaN,FIFA World Cup 2022,Al Janoub,Qatar,True
44081,26.11.2022,Poland,Saudi Arabia,NaN,NaN,FIFA World Cup 2022,Education City,Qatar,True
44082,26.11.2022,France,Denmark,NaN,NaN,FIFA World Cup 2022,Stadium 974,Qatar,True
44083,26.11.2022,Argentina,Mexico,NaN,NaN,FIFA World Cup 2022,Lusail Iconic,Qatar,True


## Get unique values

In [13]:
teams = set(data_raw.home_team.unique().tolist() + data_raw.away_team.unique().tolist())
for i, team in enumerate(teams):
    print(i, team)

0 Northern Mariana Islands
1 Sri Lanka
2 Estonia
3 Egypt
4 Zimbabwe
5 Greenland
6 Bahamas
7 Tamil Eelam
8 Runners-up Group H
9 New Zealand
10 Palau
11 Algeria
12 Panjab
13 Solomon Islands
14 Chagos Islands
15 Angola
16 Ecuador
17 Sudan
18 Winners Group C
19 Northern Cyprus
20 Eswatini
21 Faroe Islands
22 Turkey
23 Singapore
24 Gambia
25 Jordan
26 Argentina
27 Åland Islands
28 Brunei Darussalam
29 Darfur
30 Indonesia
31 Comoros
32 Moldova
33 Bonaire
34 Switzerland
35 Czech Republic
36 Antigua and Barbuda
37 American Samoa
38 North Macedonia
39 Kyrgyzstan
40 Sierra Leone
41 Netherlands
42 Winners Group A
43 Saint Pierre and Miquelon
44 Timor-Leste
45 Rhodes
46 Hungary
47 Runners-up Group G
48 Samoa
49 South Africa
50 Western Sahara
51 Niue
52 Trinidad and Tobago
53 South Ossetia
54 Winners Match 57
55 Brittany
56 Chameria
57 Kosovo
58 Tonga
59 Tajikistan
60 Yemen
61 Andalusia
62 Kenya
63 Raetia
64 Republic of St. Pauli
65 Luxembourg
66 Saint Lucia
67 Albania
68 Haiti
69 DR Congo
70 Winne

In [15]:
for col in data_raw.columns:
    if not "team" in col:
        print(col, ":", data_raw[col].unique())

date : ['1872-11-30' '1873-03-08' '1874-03-07' ... '14.12.2022' '17.12.2022'
 '18.12.2022']
home_score : [ 0.  4.  2.  3.  1.  7.  9.  5.  6.  8. 11. 13. 12. 10. 15. 14. 21. 30.
 24. 18. 16. 19. 20. 17. 22. 31. nan]
away_score : [ 0.  2.  1.  3.  4.  6.  5. 13.  8.  7. 10.  9. 11. 12. 15. 18. 16. 14.
 19. 20. 17. 21. nan]
tournament : ['Friendly' 'British Championship' 'Copa Lipton' 'Copa Newton'
 'Copa Premio Honor Argentino' 'Copa Premio Honor Uruguayo' 'Copa Roca'
 'Copa América' 'Copa Chevallier Boutell' 'Nordic Championship'
 'International Cup' 'Baltic Cup' 'Balkan Cup' 'FIFA World Cup'
 'Copa Rio Branco' 'FIFA World Cup qualification' 'Bolivarian Games'
 'CCCF Championship' 'NAFU Championship' 'Copa Oswaldo Cruz'
 'Pan American Championship' 'Copa del Pacífico' "Copa Bernardo O'Higgins"
 'AFC Asian Cup qualification' 'Atlantic Cup' 'AFC Asian Cup'
 'African Cup of Nations' 'Copa Paz del Chaco' 'Merdeka Tournament'
 'UEFA Euro qualification' 'UEFA Euro' 'Windward Islands Tourname

### Split Dataset

In [27]:
data_raw_new = data_raw[data_raw.home_score.isnull()]
display(data_raw_new)

data_raw_played = data_raw[data_raw.home_score.notnull()]
display(data_raw_played)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44060,20.11.2022,Qatar,Ecuador,NaN,NaN,FIFA World Cup 2022,Al Bayt,Qatar,False
44061,21.11.2022,England,Iran,NaN,NaN,FIFA World Cup 2022,Khalifa International,Qatar,True
44062,21.11.2022,Senegal,Netherlands,NaN,NaN,FIFA World Cup 2022,Al Thumama,Qatar,True
44063,21.11.2022,United States,Wales,NaN,NaN,FIFA World Cup 2022,Ahmad bin Ali,Qatar,True
44064,22.11.2022,Argentina,Saudi Arabia,NaN,NaN,FIFA World Cup 2022,Lusail Iconic,Qatar,True
...,...,...,...,...,...,...,...,...,...
44119,10.12.2022,Winners Match 51,Winners Match 52,NaN,NaN,FIFA World Cup 2022,Al Bayt,Qatar,NaN
44120,13.12.2022,Winners Match 57,Winners Match 58,NaN,NaN,FIFA World Cup 2022,Lusail Iconic,Qatar,NaN
44121,14.12.2022,Winners Match 59,Winners Match 60,NaN,NaN,FIFA World Cup 2022,Al Bayt,Qatar,NaN
44122,17.12.2022,Losers Match 61,Losers Match 62,NaN,NaN,FIFA World Cup 2022,Khalifa International,Qatar,NaN


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44055,27.09.2022,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False
44056,27.09.2022,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False
44057,27.09.2022,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False
44058,27.09.2022,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False


In [34]:
teams_test = set([team.strip() for team in data_raw_new.home_team.unique() if not ("group" in team.lower() or "match" in team.lower())])
print(teams_test, len(teams_test))

{'Poland', 'Japan', 'Denmark', 'Costa Rica', 'Iran', 'Australia', 'Mexico', 'Ghana', 'Switzerland', 'Morocco', 'France', 'Wales', 'Netherlands', 'Qatar', 'Argentina', 'Germany', 'Cameroon', 'Spain', 'United States', 'Tunisia', 'Croatia', 'Uruguay', 'Canada', 'England', 'South Korea', 'Serbia', 'Ecuador', 'Senegal', 'Saudi Arabia', 'Belgium', 'Brazil', 'Portugal'} 32


In [37]:
def is_test_team(match, teams):
    return match["home_team"] in teams or match["away_team"] in teams

In [41]:
data_raw_played_testonly = data_raw_played[data_raw_played.apply(lambda x: is_test_team(x, teams_test), axis=1)]
display(data_raw_played_testonly)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44045,27.09.2022,Qatar,Chile,2.0,2.0,Friendly,Vienna,Austria,True
44046,27.09.2022,Saudi Arabia,United States,0.0,0.0,Friendly,Murcia,Spain,True
44050,27.09.2022,Portugal,Spain,0.0,1.0,UEFA Nations League,Braga,Portugal,False
44051,27.09.2022,Switzerland,Czech Republic,2.0,1.0,UEFA Nations League,St. Gallen,Switzerland,False
